# 🤖 Agentic RAG Model
- subset of data, no router version

### Workflow

- 📂 **Load different ChromaDB databases**  
- 🧠 **Run with ChatGPT OSS as the backbone model for the agent**  
- 📜 **Apply router to decide which RAG agent to use**  
- 📝 **Save results**  
  - Final generated output  
  - Which search agent was used  
  - Ground truth answer for comparison


### Setup

In [1]:
!pip install chromadb
!pip install litellm
!pip install qdrant-client
!pip install -U ddgs
!pip install colab-xterm
!pip install -U langchain-ollama
!apt-get update && apt-get install -y pciutils
!pip install langchain-community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.8/20.8 MB 100.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 20.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 100.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 103.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.3/132.3 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.9/65.9 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 208.0/208.0 kB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 7.8 MB/s 

In [ ]:
!pip install -U langchain-openai
from langchain_openai import ChatOpenAI
import os
os.environ["OPENAI_API_KEY"] =

def Run(message: str) -> str:
    chat = ChatOpenAI(
        model=LLMName   # change to whichever ChatGPT model you want
#         temperature=0          # make it deterministic
    )
    response = chat([HumanMessage(content=message)])
    return response.content.strip()

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.9/81.9 kB 2.7 MB/s eta 0:00:00


In [7]:
from pathlib import Path
from google.colab import drive
import chromadb
from chromadb.config import Settings
import os
from tqdm.notebook import tqdm
import re
import json
from typing import Optional
import requests
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import HumanMessage
from langchain_core.messages import SystemMessage
import warnings
from math import ceil
import csv

warnings.filterwarnings(
    "ignore",
    category=DeprecationWarning,
    module=r"jupyter_client\.session"
)
hazards = ['Flood', 'Landslide', 'Storm', 'Tsunami', 'Hurricane', 'Wildfire','Earthquake']
drive.mount("/content/drive")
LLMName = 'gpt-5-nano'
llm = ChatOllama(model=LLMName, num_ctx=8192)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Agent Toolkits 🔍

In [ ]:
from ddgs import DDGS
from openai import OpenAI
from typing import List, Dict, Any, Iterable, Union
from sentence_transformers import CrossEncoder

# Initialize OpenAI client
os.environ["OPENAI_API_KEY"] =
client = OpenAI()

In [ ]:
# 🔍 Tool 0: LLM Hazard Agent Router
ROUTER_PROMPT = """
You are a routing expert agent. Your task is to classify a user’s question into probabilities over natural hazard categories.
These probabilities will determine which hazard expert (RAG) agent(s) are utilized (>=0.2 indicate the expert should be utilized), note to the clearly given hazard event in question (e.g. during the hurricane, the earthquake in year, or the year's storm) should be the main agent(>=0.5).

Requirements:
- Output only a valid **JSON object**.
- Use the hazard categories as keys.
- Values must be normalized probabilities that sum to 1.0.
- Do not include explanations or extra text.

Hazard categories:
["Wildfire", "Storm", "Landslide", "Hurricane", "Flood", "Earthquake","Tsunami"]

Question:
{QUESTION}

Output Example:
{{
  "Wildfire": 0.01,
  "Storm": 0.10,
  "Landslide": 0.05,
  "Hurricane": 0.61,
  "Flood": 0.21,
  "Earthquake": 0.01,
  "Tsunami":0.01
}}
"""

def extract_json(text: str):
    try:
        # grab JSON block if extra text
        match = re.search(r"\{.*\}", text, re.DOTALL)
        if not match:
            return None
        data = json.loads(match.group())
        return data
    except Exception:
        return None

def validate_probs(data: dict):
    if not data:
        return False
    # all hazards present
    if set(data.keys()) != set(hazards):
        return False
    # all values are floats
    try:
        vals = [float(data[h]) for h in hazards]
    except Exception:
        return False
    # normalize check
    return abs(sum(vals) - 1.0) < 1e-3

def hazard_router(question: str, max_retries: int = 50):
    for attempt in range(max_retries):
        prompt = ROUTER_PROMPT.format(QUESTION=question)
        resp = llm.invoke(prompt).content
        data = extract_json(resp)
        if validate_probs(data):
            hazards = [haz for haz, prob in data.items() if prob >= 0.2]
            return hazards
        print(f"Retry {attempt+1}/{max_retries} failed, retrying...")
    raise RuntimeError("Failed to extract valid hazard probabilities after retries")

In [ ]:
# 🔍 Tool 1: Oneline Search
def format_search_results(results):
    return "\n\n".join(doc["body"] for doc in results)

question = 'What is the SSHS category of 2017 Hurricane Harvey?'
results = DDGS().text(question, max_results=2)
results

In [ ]:
# 🔍 Tool 2: RAG agents

# Embedding
def embed(query: str) -> List[float]:
    response = client.embeddings.create(
        model="text-embedding-3-large",
        input=query
    )
    return response.data[0].embedding

# Cross Encoder & Reranking
_CROSS_ENCODER_MODEL_NAME = "cross-encoder/mmarco-mMiniLMv2-L12-H384-v1"
_cross_encoder_singleton: CrossEncoder = None

def _get_cross_encoder() -> CrossEncoder:
    global _cross_encoder_singleton
    if _cross_encoder_singleton is None:
        _cross_encoder_singleton = CrossEncoder(_CROSS_ENCODER_MODEL_NAME)
    return _cross_encoder_singleton

def rerank(query: str, retrieved_chunks: List[str], top_k: int) -> List[str]:
    """
    Cross-encoder reranking. Returns top_k chunks sorted by relevance.
    """
    if not retrieved_chunks:
        return []
    cross_encoder = _get_cross_encoder()
    pairs = [(query, chunk) for chunk in retrieved_chunks]
    scores = cross_encoder.predict(pairs)  # higher is better

    # Pair each chunk with its score, sort descending, take top_k
    scored_chunks = sorted(zip(retrieved_chunks, scores), key=lambda x: x[1], reverse=True)
    return [chunk for chunk, _ in scored_chunks[:top_k]]

# Bi-encoder retrieval
def _retrieve_bi(collection: chromadb.Collection, query_embedding: List[float], k: int) -> List[str]:
    """
    Retrieve top-k documents from a Chroma collection using the bi-encoder embedding.
    Returns: list[str] of document texts.
    """
    res = collection.query(
        query_embeddings=[query_embedding],  # list of one embedding
        n_results=k
    )
    # Chroma returns 'documents' as List[List[str]] (one list per query)
    docs_nested = res.get("documents") or []
    return (docs_nested[0] if docs_nested else [])  # flatten the single-query case

## Agentic RAG Framework 🤖

#### Main loop Functions

In [9]:
def load_json_items(path: Path) -> List[Dict[str, Any]]:
    """
    Load items from either:
      - JSON Lines (one JSON object per line), or
      - a single JSON array file.
    Returns a list of dicts.
    """
    txt = path.read_text(encoding="utf-8").strip()
    items: List[Dict[str, Any]] = []
    # Try JSON array first
    try:
        data = json.loads(txt)
        if isinstance(data, list):
            items = data
        elif isinstance(data, dict):
            items = [data]
        else:
            raise ValueError("Unsupported JSON top-level type.")
        return items
    except Exception:
        pass
    # Fallback: treat as JSONL
    for line in txt.splitlines():
        line = line.strip()
        if not line:
            continue
        items.append(json.loads(line))
    return items

In [10]:
# --------------------
# Model runner
# --------------------
# --------------------
# Prompt templates
# --------------------
TF_PROMPT = (
    "You are an expert in hazards and resilience, evaluating the historical hazard: {disaster_name}.\n\n"
    "====================\n"
    "STATEMENT TO JUDGE:\n"
    "{statement}\n"
    "====================\n\n"
    "Related excerpts:\n{joined_chunks}\n\n"
    "Your task: Judge the statement strictly based on the excerpts of information provided by renaissance reports.\n"
    "====================\n"
    "STATEMENT TO JUDGE:\n"
    "{statement}\n"
    "====================\n\n"
    "Answer with ONLY one word: true OR false\n"
    "No punctuation, no explanation, no extra words."
)


MC_PROMPT = (
    "You are an expert in hazards and resilience, evaluating the historical hazard: {disaster_name}.\n\n"
    "====================\n"
    "QUESTION:\n"
    "{question}\n\n"
    "OPTIONS:\n"
    "{options_block}\n"
    "====================\n\n"
    "Related excerpts:\n{joined_chunks}\n\n"
    "Your task: Choose the single BEST answer to the question strictly based on the excerpts provided by renaissance reports.\n"
    "====================\n"
    "QUESTION:\n"
    "{question}\n\n"
    "OPTIONS:\n"
    "{options_block}\n"
    "====================\n\n"
    "Answer with ONLY one letter: A B C or D\n"
    "No punctuation, no explanation, no extra words."
)

In [11]:
# --------------------
# Prompt templates
# --------------------
TF_PROMPT_no_info = (
    "You are an expert in hazards and resilience, evaluating the historical hazard: {disaster_name}.\n\n"
    "====================\n"
    "STATEMENT TO JUDGE:\n"
    "{statement}\n"
    "====================\n\n"
    "Your task: Judge the statement considering the excerpts of information.\n"
    "====================\n"
    "STATEMENT TO JUDGE:\n"
    "{statement}\n"
    "====================\n\n"
    "Answer with ONLY one word: true OR false\n"
    "No punctuation, no explanation, no extra words."
)

MC_PROMPT_no_info = (
    "You are an expert in hazards and resilience, evaluating the historical hazard: {disaster_name}.\n\n"
    "====================\n"
    "QUESTION:\n"
    "{question}\n\n"
    "OPTIONS:\n"
    "{options_block}\n"
    "====================\n\n"
    "Your task: Choose the single BEST answer to the question.\n"
    "====================\n"
    "QUESTION:\n"
    "{question}\n\n"
    "OPTIONS:\n"
    "{options_block}\n"
    "====================\n\n"
    "Answer with ONLY one letter: A B C or D\n"
    "No punctuation, no explanation, no extra words."
)

In [12]:
def normalize_tf(ans: str) -> str:
    """
    Normalize model output to 'true' or 'false' strictly.
    Falls back to regex keyword detection.
    """
    s = ans.strip().lower()
    if s in {"true", "false"}:
        return s
    # Heuristics
    if "true" in s and "false" not in s:
        return "true"
    if "false" in s and "true" not in s:
        return "false"
    # If ambiguous, keep first token
    return "true" if s.startswith("t") else "false"

def normalize_mc(ans: str) -> str:
    """
    Normalize model output to a single uppercase letter A-D.
    """
    s = ans.strip().upper()
    # Grab first A/B/C/D we can find
    m = re.search(r"\b([ABCD])\b", s)
    if m:
        return m.group(1)
    # Fallback: first character if valid
    if s[:1] in {"A", "B", "C", "D"}:
        return s[:1]
    # Last resort: A
    return "A"

def options_to_block(options: Iterable[str]) -> str:
    """
    Ensure options appear one per line as 'A. ...' etc.
    If options are already labeled, keep them; else auto-label.
    """
    opts = list(options)
    labeled = all(re.match(r"^[ABCD]\s*\.", o.strip()) for o in opts)
    if labeled:
        return "\n".join(o if o.endswith("\n") else o for o in opts)
    labels = ["A", "B", "C", "D"]
    return "\n".join(f"{labels[i]}. {opts[i]}" for i in range(min(4, len(opts))))

#### Main Loop

In [20]:
from pathlib import Path
from collections import Counter
import json

def load_json_items(path):
    with open(path, "r", encoding="utf-8") as f:
        data = json.load(f)
    # handle both list and dict-based formats
    if isinstance(data, dict) and "data" in data:
        return data["data"]
    return data

categories = ['Flood', 'Landslide', 'Storm', 'Tsunami', 'Hurricane', 'Wildfire', 'Earthquake']
QA_folder = Path("/content/drive/MyDrive/LLMs/LLM_Hazards/New_QA_Cate")

for category in categories:
    QA_path = QA_folder / category

    total_count = 0
    qtype_counter = Counter()
    cate1_counter = Counter()
    combo_counter = Counter()   # category1 × qtype joint count

    for json_path in sorted(QA_path.glob("*.json")):
        items = load_json_items(json_path)
        for obj in items:
            if obj is None:
                continue

            qtype = obj.get("question_type", "").strip().lower()
            category1 = obj.get("category", "").strip()
            if category1 == "Invalid":
                continue

            # Count overall and per type
            if qtype in ["true_false", "mc_single"]:
                total_count += 1
                qtype_counter[qtype] += 1
                cate1_counter[category1] += 1
                combo_counter[(category1, qtype)] += 1

    print(f"\n=== {category} ===")
    print(f"Total Valid QA: {total_count}")



=== Flood ===
Total Valid QA: 658

=== Landslide ===
Total Valid QA: 263

=== Storm ===
Total Valid QA: 348

=== Tsunami ===
Total Valid QA: 70

=== Hurricane ===
Total Valid QA: 891

=== Wildfire ===
Total Valid QA: 92

=== Earthquake ===
Total Valid QA: 3454


In [15]:
# --------------------
# Main loop
# --------------------
categories = ['Flood', 'Landslide', 'Storm', 'Tsunami', 'Hurricane', 'Wildfire','Earthquake']
QA_folder = Path("/content/drive/MyDrive/LLMs/LLM_Hazards/New_QA_Cate")

for category in categories:
    QA_path = QA_folder / category
    # Iterate all .json files
    # Counter
    count = 0
    for json_path in sorted(QA_path.glob("*.json")):
        items = load_json_items(json_path)
        rows = []
        for obj in items:
            if obj is None:
              continue
            qtype = obj.get("question_type", "").strip().lower()
            qa = obj.get("question_answer", {})
            disaster_type = obj.get("disaster_type", "")
            category1 = obj.get("category", "")
            disaster_name = obj.get("disaster_name", "")

            if category1 == "Invalid":
                continue

            if qtype == "true_false":
                count = count +1

            elif qtype == "mc_single":
                if category1 == "Invalid":
                    continue
                count = count +1
    print(f"{category} Count: {count}")

Flood Count: 658
Landslide Count: 263
Storm Count: 348
Tsunami Count: 70
Hurricane Count: 891
Wildfire Count: 92
Earthquake Count: 3454


In [ ]:
# --------------------
# Main loop
# --------------------
categories = ['Flood', 'Landslide', 'Storm', 'Tsunami', 'Hurricane', 'Wildfire','Earthquake']
QA_folder = Path("/content/drive/MyDrive/LLMs/LLM_Hazards/New_QA_Cate")

for category in categories:
    QA_path = QA_folder / category
    # Iterate all .json files
    # Counter
    n=0
    for json_path in sorted(QA_path.glob("*.json")):
        n=n+1
        if n>4:
          continue
        items = load_json_items(json_path)
        rows = []
        for obj in items:
            if obj is None:
              continue
            qtype = obj.get("question_type", "").strip().lower()
            qa = obj.get("question_answer", {})
            disaster_type = obj.get("disaster_type", "")
            category1 = obj.get("category", "")
            disaster_name = obj.get("disaster_name", "")

            if category1 == "Invalid":
                print(f"Skipping invalid category: {category1}")
                continue

            if qtype == "true_false":
                statement = qa.get("statement", "")
                gt = qa.get("answer", None)  # True/False (bool) if provided
                rag_query = f"{disaster_name} {statement}"

                # Router + RAG Agents
                joined_chunks = []
                hazard_list = []

                # Define values
                is_enough_online = '-1'
                is_enough_rag = '-1'

                message = TF_PROMPT_no_info.format(disaster_name = disaster_name,statement=statement)

                raw = Run(message)
                pred = normalize_tf(raw)
                print(pred,str(gt))

                is_correct = None
                if isinstance(gt, bool):
                    is_correct = (pred == str(gt).lower())

                rows.append({
                    "disaster_type": disaster_type,
                    "disaster_name": disaster_name,
                    "category": category,
                    "category1": category1,
                    "file": json_path.name,
                    "question_type": "true_false",
                    "question_or_statement": statement,
                    "options": "",
                    "router":str(hazard_list),
                    "retreived_chunks": joined_chunks,
                    "ground_truth": str(gt).lower() if isinstance(gt, bool) else "",
                    "model_raw": raw,
                    "model_pred": pred,
                    "is_correct": is_correct,
                    "is_enough_rag": is_enough_rag,
                    "is_enough_online": is_enough_online
                })

            elif qtype == "mc_single":
                question = qa.get("question", "")
                options = qa.get("options", [])
                correct = qa.get("correct", "")  # e.g., "A"
                options_block = options_to_block(options)
                category1 = obj.get("category", "")
                if category1 == "Invalid":
                    print(f"Skipping invalid category: {category1}")
                    continue
                # Retrieval and Rerank here
                rag_query = f"{disaster_name} {question}"

                # Router + RAG Agents
                joined_chunks = []
                hazard_list = []

                # Define values
                is_enough_online = '-1'
                is_enough_rag = '-1'


                message = MC_PROMPT_no_info.format(disaster_name = disaster_name,question=question,options_block = options_block)

                # print(message)

                raw = Run(message)
                pred = normalize_mc(raw)
                print(pred,correct)

                is_correct = (pred == correct.strip().upper()) if correct else None

                rows.append({
                    "disaster_type": disaster_type,
                    "disaster_name": disaster_name,
                    "category": category,
                    "category1": category1,
                    "file": json_path.name,
                    "question_type": "mc_single",
                    "question_or_statement": question,
                    "options": " | ".join(options),
                    "router":str(hazard_list),
                    "retreived_chunks": joined_chunks,
                    "ground_truth": correct.strip().upper() if correct else "",
                    "model_raw": raw,
                    "model_pred": pred,
                    "is_correct": is_correct,
                    "is_enough_rag": is_enough_rag,
                    "is_enough_online": is_enough_online
                })

            else:
                # Unknown type: skip gracefully
                rows.append({
                    "disaster_type": disaster_type,
                    "disaster_name": disaster_name,
                    "category": category,
                    "category1": category1,
                    "file": json_path.name,
                    "question_type": qtype,
                    "question_or_statement": "",
                    "options": "",
                    "router":str(hazard_list),
                    "retreived_chunks": joined_chunks,
                    "ground_truth": "",
                    "model_raw": "",
                    "model_pred": "",
                    "is_correct": None,
                    "is_enough_rag": None,
                    "is_enough_online": None
                })

        # Save answers to CSV in the corresponding model/category output folder
        output_folder = Path("/content/drive/MyDrive/LLMs/LLM_Hazards/Final_Experiments/Outputs/Output_zero_shot_llama")
        out_dir = output_folder / category
        out_dir.mkdir(parents=True, exist_ok=True)
        out_csv = out_dir / (json_path.stem + ".csv")

        fieldnames = [
            "disaster_type", "disaster_name", "category","category1", "file",
            "question_type", "question_or_statement", "options","router","retreived_chunks",
            "ground_truth", "model_raw", "model_pred", "is_correct","is_enough_rag","is_enough_online"
        ]

        with out_csv.open("w", newline="", encoding="utf-8") as f:
            writer = csv.DictWriter(f, fieldnames=fieldnames)
            writer.writeheader()
            for r in rows:
                writer.writerow(r)

        print(f"Saved: {out_csv}")

### Result performance check

In [ ]:
! pip install tabulate

In [ ]:
import csv
from pathlib import Path
from collections import defaultdict

# Set the root output folder where per-file CSVs were written
root = output_folder

def _to_bool(s):
    return str(s).strip().lower() in ("true", "1", "yes")

def _safe_float(num, den):
    return (num / den) if den else 0.0

# Collect all rows from all CSVs under the (optional) category subfolder you just used.
# If you want to aggregate **only** one category, use: for p in (root / category).glob("*.csv")
all_rows = []
for p in root.rglob("*.csv"):
    # skip any *_summary.csv if you created those earlier
    if p.name.endswith("_summary.csv"):
        continue
    with p.open("r", encoding="utf-8", newline="") as f:
        reader = csv.DictReader(f)
        for r in reader:
            all_rows.append(r)

In [ ]:
from tabulate import tabulate  # pip install tabulate if not installed

def _to_bool(s):
    return str(s).strip().lower() in ("true", "1", "yes")

n = len(rows) or 1
acc = sum(1 for r in rows if r.get("is_correct") is True) / n
rag_rate = sum(1 for r in rows if r.get("is_enough_rag") == "1") / n
online_rate = sum(1 for r in rows if r.get("is_enough_online") == "1") / n
both_insuff_rate = sum(
    1 for r in rows
    if r.get("is_enough_rag") != "1" and r.get("is_enough_online") == "0"
) / n

# Confusion matrix
eval_rows = [r for r in rows if str(r.get("ground_truth","")).strip() != ""]
tp = fp = fn = tn = 0
for r in eval_rows:
    gt = _to_bool(r.get("ground_truth", ""))
    pred = _to_bool(r.get("model_pred", ""))
    if pred and gt: tp += 1
    elif pred and not gt: fp += 1
    elif not pred and gt: fn += 1
    else: tn += 1

# ---- Print summary ----
print("\n=== Summary Metrics ===")
print(tabulate([
    ["Total samples", n],
    ["Accuracy", f"{acc:.3f}"],
    ["RAG sufficiency rate", f"{rag_rate:.3f}"],
    ["Online sufficiency rate", f"{online_rate:.3f}"],
    ["Both insufficient rate", f"{both_insuff_rate:.3f}"],
], headers=["Metric", "Value"], tablefmt="github"))

print("\n=== Confusion Matrix (GT vs Pred) ===")
print(tabulate([
    ["TP", tp, "GT=True, Pred=True"],
    ["FP", fp, "GT=False, Pred=True"],
    ["FN", fn, "GT=True, Pred=False"],
    ["TN", tn, "GT=False, Pred=False"],
], headers=["Cell", "Count", "Meaning"], tablefmt="github"))

In [ ]:
import csv
from pathlib import Path
from collections import defaultdict

# Set the root output folder where per-file CSVs were written
root = output_folder

def _to_bool(s):
    return str(s).strip().lower() in ("true", "1", "yes")

def _safe_float(num, den):
    return (num / den) if den else 0.0

# Collect all rows from all CSVs under the (optional) category subfolder you just used.
# If you want to aggregate **only** one category, use: for p in (root / category).glob("*.csv")
all_rows = []
for p in root.rglob("*.csv"):
    # skip any *_summary.csv if you created those earlier
    if p.name.endswith("_summary.csv"):
        continue
    with p.open("r", encoding="utf-8", newline="") as f:
        reader = csv.DictReader(f)
        for r in reader:
            all_rows.append(r)

# --- Overall metrics ---
n = len(all_rows)
correct = sum(1 for r in all_rows if r.get("is_correct") in (True, "True", "true", "1"))
acc = _safe_float(correct, n)

rag_sufficient = sum(1 for r in all_rows if r.get("is_enough_rag") == "1")
online_sufficient = sum(1 for r in all_rows if r.get("is_enough_online") == "1")
both_insufficient = sum(1 for r in all_rows if r.get("is_enough_rag") != "1" and r.get("is_enough_online") == "0")

rag_rate = _safe_float(rag_sufficient, n)
online_rate = _safe_float(online_sufficient, n)
both_insuff_rate = _safe_float(both_insufficient, n)

# --- Confusion matrix (only rows that have ground_truth) ---
eval_rows = [r for r in all_rows if str(r.get("ground_truth", "")).strip() != ""]
tp = fp = fn = tn = 0
for r in eval_rows:
    gt = _to_bool(r.get("ground_truth", ""))
    pred = _to_bool(r.get("model_pred", ""))
    if pred and gt:
        tp += 1
    elif pred and not gt:
        fp += 1
    elif not pred and gt:
        fn += 1
    else:
        tn += 1

# --- Per-category accuracy ---
by_cat = defaultdict(lambda: {"n": 0, "correct": 0})
for r in all_rows:
    cat = r.get("category", "UNKNOWN")
    by_cat[cat]["n"] += 1
    if r.get("is_correct") in (True, "True", "true", "1"):
        by_cat[cat]["correct"] += 1

# ---- Print summary ----
print("\n=== Overall Summary ===")
print(f"Total samples: {n}")
print(f"Accuracy: {acc:.4f}")
print(f"RAG sufficiency rate: {rag_rate:.4f}")
print(f"Online sufficiency rate: {online_rate:.4f}")
print(f"Both insufficient rate: {both_insuff_rate:.4f}")

print("\n=== Confusion Matrix (GT vs Pred) ===")
print(f"TP: {tp} | FP: {fp} | FN: {fn} | TN: {tn}")
total_eval = tp + fp + fn + tn
if total_eval:
    precision = tp / (tp + fp) if (tp + fp) else 0.0
    recall = tp / (tp + fn) if (tp + fn) else 0.0
    f1 = (2 * precision * recall / (precision + recall)) if (precision + recall) else 0.0
    print(f"Precision: {precision:.4f} | Recall: {recall:.4f} | F1: {f1:.4f}")

print("\n=== Per-Category Accuracy ===")
for cat, d in sorted(by_cat.items(), key=lambda x: x[0]):
    cat_acc = _safe_float(d["correct"], d["n"])
    print(f"- {cat}: {cat_acc:.4f} ({d['correct']}/{d['n']})")
